In [15]:
import pandas as pd
import numpy as np
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from helpers import *
import copy

# Changes:
# 1. Start with population including greedy cycle and greedy regret results
# 2. Change the random operator to common edges + common nodes + greedy cycle
# 3. Add diversification mechanism - randomly perturb the solution with probability accoring to temperature

# improvements:
# - as an initial population, use some of the previous methods (e.g. greedy cycle, greedy regret) - they are fast and will give a good starting point, further improved LS.
# change the operators, add new one
# diversification machanism


# combine random introduces too much randomness and it makes the method run LS for much too long

# TODO add new recombination operator not introducing as much randomness
# common edges, then make subpaths from common nodes and insert them randomly, then use greedy cycle to repair the solution

# TODO Start initial population from greedy cycle, greedy regret, and the rest as before

# TODO add diversification mechanism
# randomly perturb the solution with probability accoring to temperature

In [21]:
def greedy_cycle(path, D, costs):
    path = list(path)
    target_length = math.ceil(len(D) / 2)
    while len(path) < target_length:
        closest_node = 0
        best_increase = MAX_DIST
        free_nodes = [x for x in range(D.shape[0]) if x not in path]
        for idx in range(len(path)):
            for new_node in free_nodes:
                if idx == 0:
                    prev_node = path[-1]
                    next_node = path[0]
                else:
                    prev_node = path[idx - 1]
                    next_node = path[idx]
                curr_increase = (
                    D[prev_node, new_node]
                    + D[new_node, next_node]
                    - D[prev_node, next_node]
                    + costs[new_node]
                )
                if curr_increase < best_increase:
                    best_increase = curr_increase
                    closest_node = new_node
                    node_pos = idx
        path.insert(node_pos, closest_node)
    return np.array(path)

def greedy_regret_repair(path, nodes_available,  D, costs, weights=0.5):
    target_length = math.ceil(len(D) / 2)
    path = list(path) + [path[0]]
    edges = []
    nodes_available = [x for x in range(len(D)) if x not in path]
    for idx, node in enumerate(path):
        if idx == len(path)-1: break
        edges.append([node, path[(idx+1) % len(path)]])
    while len(path) < target_length+1 and len(nodes_available) > 0:
        M = np.zeros((len(nodes_available), len(edges)))
        indices = np.array(nodes_available, dtype=int)
        for edge_ix in range(len(edges)):
            a, b = edges[edge_ix]
            var = D[a, :] + D[:, b] - D[a, b] + costs
            M[:,edge_ix] = var[indices]
        best_score = -MAX_DIST
        replaced_edge = 0
        best_node = 0
        for node_idx in range(len(nodes_available)):
            best, second_best = np.partition(M[node_idx], 1)[:2]
            regret = second_best - best
            score = weights * regret - (1 - weights) * np.min(M[node_idx])
            if score > best_score:
                best_score = score
                replaced_edge = np.argmin(M[node_idx])
                best_node = nodes_available[node_idx]
        path.insert(replaced_edge+1, best_node)
        nodes_available.remove(best_node)
        a, b = edges[replaced_edge]
        edges.pop(replaced_edge)
        edges.insert(replaced_edge, [a, best_node])
        edges.insert(replaced_edge + 1, [best_node, b])
    return path[:-1]

def get_edges(path):
    edges = []
    for idx, node in enumerate(path):
        edge = [node, path[(idx+1) % len(path)]]
        edges.append(edge)
    return edges

def get_common_edges(path1, path2):
    common_edges = []
    for idx, edge_start in enumerate(path1):
        edge_end = path1[(idx+1) % len(path1)]
        if edge_start in path2:
            e_idx = path2.index(edge_start)
            if edge_end == path2[(e_idx+1) % len(path2)]:
                edge = [edge_start, edge_end]
                common_edges.append(edge)
            elif edge_end == path2[(e_idx-1) % len(path2)]:
                edge = [edge_start, edge_end]
                common_edges.append(edge)
    return common_edges


def get_initial_edges_path(path1, path2):
    offspring = []
    edges = get_edges(path1)
    common_edges = get_common_edges(path1, path2)
    for edge in edges[1:]:
        if edge in common_edges or edge[::-1] in common_edges:
            if len(offspring) > 0:
                if offspring[-1] != edge[0]:
                    offspring.append(edge[0])
                if offspring[0] != edge[1]:
                    offspring.append(edge[1])
            else:
                offspring.extend(edge)
    return offspring

def combine_new(path1, path2, D, costs):
    common_nodes = [node for node in path1 if node in path2]
    offspring = get_initial_edges_path(path1, path2)
    nodes_available = [x for x in common_nodes if x not in offspring]
    offspring.extend(nodes_available)
    offspring = greedy_cycle(offspring, D, costs)
    return offspring

def combine_heuristic(path1, path2, D, costs):
    offspring = get_initial_edges_path(path1, path2)
    if len(offspring) == 0:
        offspring.extend([path1[0], path1[1]])
    common_nodes = [node for node in path1 if node in path2 and node not in offspring]
    repaired = greedy_regret_repair(offspring, common_nodes, D, costs)
    nodes_available = [x for x in range(len(D)) if x not in repaired]
    repaired = greedy_regret_repair(repaired, nodes_available, D, costs)
    return repaired

def perturbation_swap5(D, cycle, n=10):
    og_nodes = set(cycle)
    nodes = set(list(range(D.shape[0])))
    unused = nodes - og_nodes
    copy_cycle = copy.deepcopy(cycle)
    for i in range(n):
        random_unused = random.choice(list(unused))
        unused = unused - set([random_unused])
        random_index = random.choice(list(range(len(cycle))))
        copy_cycle[random_index] = random_unused
    return copy_cycle

def perturbation_replacenn(D, costs, path, n=10):
    random_indices = random.sample(list(range(len(path))), n)
    copy_cycle = copy.deepcopy(path)
    for i in range(n):
        next_index = (random_indices[i] + 1) % len(path)
        distances = copy.deepcopy(D[copy_cycle[random_indices[i]], :]) + costs
        distances[copy_cycle] = MAX_DIST
        copy_cycle[next_index] = np.argmin(distances)
    return copy_cycle

def diversify(population, scores, D, costs):
    # select 10 percent of population with worst scores from the population, perform a perturbation on them, and apply LS
    # if the score is better than the worst score, replace it

    # select 10 percent
    # print('Diversify')
    worst_idx = np.argsort(scores)[-int(len(population) * 0.2):]
    worst = [population[idx] for idx in worst_idx]
    worst_scores = [scores[idx] for idx in worst_idx]
    for idx in range(len(worst)):
        perturbed = perturbation_swap5(D, worst[idx])
        perturbed = list(local_search(perturbed, costs, D))
        perturbed_score = evaluate(D, perturbed, costs)
        if perturbed_score < worst_scores[idx]:
            population[worst_idx[idx]] = perturbed
            scores[worst_idx[idx]] = perturbed_score
    return population, scores


def HEA(D, costs, LS_enable=False,  POP_SIZE = 20, time_limit=200):
    counter = 0
    population = []
    scores = []
    cycle = greedy_cycle(random_sequence(D), D, costs)
    regret = greedy_regret_repair(list(cycle[:3]), [x for x in range(len(D)) if x not in cycle], D, costs)
    population.append(list(regret))
    population.append(list(cycle))
    while len(population) < POP_SIZE:
        path = random_sequence(D)
        path = list(local_search(path, costs, D))
        score = evaluate(D, path, costs)
        if score not in scores:
            population.append(list(path))
            scores.append(score)
    time_start = time.time()
    c = 0
    while True:
        counter += 1
        c += 1
        parents = random.choices(population, k=2)
        if random.random() < 0.5:
            offspring = list(combine_new(parents[0], parents[1], D, costs))
            score = evaluate(D, offspring, costs)
        else:
            offspring = combine_heuristic(parents[0], parents[1], D, costs)
            score = evaluate(D, offspring, costs)
        if LS_enable:
            offspring = list(local_search(offspring, costs, D))
        offspring_score = evaluate(D, offspring, costs)
        worst_idx = scores.index(max(scores))
        if c > 20:
            population, scores = diversify(population, scores, D, costs)
            c = 0
        if offspring_score < scores[worst_idx]:
            if offspring not in population and offspring_score not in scores:
                c = 0
                # print(offspring_score)
                population[worst_idx] = offspring
                scores[worst_idx] = offspring_score
        if time.time() - time_start > time_limit:
            break
    best_found = population[scores.index(min(scores))]
    return best_found, counter

nodes, costs, D = read_data('TSPA.csv')
HEA(D, costs, LS_enable=True,  POP_SIZE=20, time_limit=200)

75055
74469
74507
74881
75052
74727
74789
74609
75138
75244
74441
74244
73914
76094
74677
75212
74681
75128
74651
74303
74238
74903
74689
73847
74893
74410
74437
73791
74260
74541
74510
74470
74403
74262
74371
74473
73818
73917
73426
73726
73494
74335
73965
73820
74400
73749
74257
73863
73751
74129
74178
73921
73881
73767
73770
73530
73414
73795
73642
73760
73275
73653
73829
73779
73313
73618
73412
73776
73705
73439
73473
73233
73659
73587
73695
73464
73633
73533
73375
73516
73547
73249
73592
73198
73393
73475
73330
73352
73199
73436
73203
73164
73318
73363
73333
73314
73229
73337
73254
73258
73059
73130
Diversify
73097
73175
Diversify
73072
Diversify
73165
73145
73143
73220
73255
73224
Diversify
73074
73131
73054
Diversify
Diversify
73027
73008
73192
73155
73042
73122
73149
Diversify
Diversify
73154
Diversify
73094
Diversify
73052
Diversify
73018
Diversify
73013
Diversify
73064
73028
73047
Diversify
Diversify
73096
Diversify
Diversify


([192,
  199,
  41,
  177,
  1,
  75,
  189,
  109,
  119,
  130,
  152,
  11,
  160,
  106,
  48,
  92,
  26,
  8,
  124,
  80,
  14,
  111,
  89,
  12,
  94,
  98,
  156,
  172,
  6,
  66,
  190,
  72,
  73,
  31,
  95,
  169,
  112,
  51,
  135,
  99,
  101,
  167,
  45,
  186,
  127,
  88,
  153,
  161,
  76,
  145,
  128,
  132,
  36,
  55,
  22,
  117,
  15,
  108,
  171,
  21,
  194,
  79,
  87,
  141,
  144,
  154,
  81,
  180,
  32,
  62,
  53,
  195,
  113,
  74,
  163,
  61,
  71,
  20,
  64,
  185,
  116,
  27,
  147,
  96,
  59,
  143,
  159,
  164,
  178,
  19,
  0,
  149,
  50,
  91,
  121,
  43,
  77,
  4,
  114,
  175],
 791)

In [3]:
def evaluate_solution(filename, D, costs, LS_enable=False,  POP_SIZE = 20, time_limit=200):
    start = time.time()
    path, counter = HEA(D, costs, LS_enable=False,  POP_SIZE=POP_SIZE, time_limit=time_limit)
    end = time.time()
    score = evaluate(D, path, costs)
    return {
        'Filename': filename,
        'Version': 'LS' if LS_enable else 'no LS',
        'Iterations': counter,
        'Path': path,
        'Score': score,
        'Time': (end - start)
    }

# %%
# result = evaluate_solution('TSPA.csv', D, costs)
# result

# %%
if __name__ == "__main__":
    test_start = time.time()
    files = ['TSPA.csv', 'TSPB.csv', 'TSPC.csv', 'TSPD.csv']
    for filename in files:
        prefix = "HEA"
        mapping_out_files= {
            'TSPA.csv' : f'{prefix}_TSPA_out.csv',
            'TSPB.csv' : f'{prefix}_TSPB_out.csv',
            'TSPC.csv' : f'{prefix}_TSPC_out.csv',
            'TSPD.csv' : f'{prefix}_TSPD_out.csv'
        }
        nodes, costs, D = read_data(filename)
        num_iterations = 20
        time_limit = 200
        results = Parallel(n_jobs=-1)(delayed(evaluate_solution)(filename, D, costs, LS_enable=True, POP_SIZE=20, time_limit=time_limit)
                                    for i in range(num_iterations)
                                    )

        results_df = pd.DataFrame(results)
        results_df.to_csv(mapping_out_files[filename])
        test_end = time.time()
        print(test_end - test_start)

KeyboardInterrupt: 